In [1]:
# import some useful packages
import matplotlib.pyplot as plt
import numpy as np
import cv2
%matplotlib inline

In [2]:
import numpy as np
import cv2

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, vertices, 255)
    
    return cv2.bitwise_and(img, mask)

In [3]:
def canny(img,low,high):
    return cv2.Canny(img,low,high)

def blur(img,kernel_size):
    return cv2.GaussianBlur(img,(kernel_size,kernel_size),0)


def improved_pipeline(img):
    height, width = img.shape[:2]
    
    vertices = np.array([[(50, height),
                          (width/2-45, height/2+60), 
                          (width/2+45, height/2+60), 
                          (width-50,height)]], dtype='int')

    img_gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img_blur=blur(img_gray,3)
    img_canny=canny(img_blur,50,150)
    img_bitwise=region_of_interest(img_canny,vertices)
    lines=cv2.HoughLinesP(img_bitwise,1,np.pi/360,70,1,10)
    lines_r=np.zeros(((len(lines),1,4)),np.int32)###조건에 충족하는 선분만 담는 배열
    re_lines=np.zeros((2,1,4),np.int32)###결과배열 담을 곳.
    a=True
    b=True
    for i in range(len(lines)):
        if(not(lines[i,0,1]-lines[i,0,3]==0 or lines[i,0,0]-lines[i,0,2]==0)):
            #print("------",lines[i,0,1]-lines[i,0,3]/lines[i,0,0]-lines[i,0,2])
            if(abs((lines[i,0,1]-lines[i,0,3])/(lines[i,0,0]-lines[i,0,2]))>0.5):###각 선분의 기울기를 검사하여 조건을 충족하는 것만 표시
                lines_r[i,0,0]=lines[i,0,0]
                lines_r[i,0,1]=lines[i,0,1]
                lines_r[i,0,2]=lines[i,0,2]
                lines_r[i,0,3]=lines[i,0,3]
                #print("----------",(lines[i,0,1]-lines[i,0,3])/(lines[i,0,0]-lines[i,0,2]))
                if((lines[i,0,1]-lines[i,0,3])/(lines[i,0,0]-lines[i,0,2])>0):###기울기가 양수이면
                    if(a==True):
                        re_lines[0,0,0]=lines_r[i,0,0]
                        re_lines[0,0,1]=lines_r[i,0,1]#첫 시작점의 x1,y1만 저장
                        a=False
                    else:
                        re_lines[0,0,2]=lines_r[i,0,2]
                        re_lines[0,0,3]=lines_r[i,0,3]#계속 갱신될 것임 -> 가장 마지막 x2,y2만 저장
                else:###기울기가 음수이면
                    if(b==True):
                        re_lines[1,0,0]=lines_r[i,0,0]
                        re_lines[1,0,1]=lines_r[i,0,1]#첫 시작점의 x1,y1만 저장
                        b=False
                    else:
                        re_lines[1,0,2]=lines_r[i,0,2]
                        re_lines[1,0,3]=lines_r[i,0,3]#계속 갱신될 것임 -> 가장 마지막 x2,y2만 저장
    #print(lines_r)
    print("result : ",re_lines)
        
    for i in range(len(re_lines)):
        for x1,y1,x2,y2 in re_lines[i]:
            result=cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)
    
    return result

In [5]:
listOfFiles = ['solidWhiteRight.jpg', 'solidYellowLeft.jpg', 'challenge.jpg']

for i, file in enumerate(listOfFiles):
    img = cv2.imread(file)
    
    result = improved_pipeline(img)
    cv2.imshow('lane detection', result)
    cv2.waitKey(0) 
    
cv2.destroyAllWindows()

result :  [[[557 357 832 538]]

 [[310 423 344 407]]]
result :  [[[771 484 752 483]]

 [[211 490 417 347]]]
result :  [[[680 418 759 462]]

 [[410 379 312 461]]]
